<img src = '../docs/qdart.png' height = '400px'>

- qdas search thì sẽ trả về 1 mạch u nhưng mình sẽ đặt vấn đề là có cần concat mạch u vào mạch cũ hay ko?
- action của policy gradient có thể là 0 hoặc 1, có concat hoặc ko concat
- state s_t của policy gradient là circuit_old, s_t+1 là circuit mới
- nhưng hàm policy pi thì sẽ là 1 neural network pi(s_t) để encode state
- Mỗi state, dqas dùng hàm phân bố p(k=j,a) để chọn layer tạo mạch
- nhưng em xem thì dùng shapeley value có vẻ hiệu quả và phù hợp với context này (vì tham số a của qdas hiện giờ nó ko phụ thuộc vào cấu trúc cổng trong mạch và phụ thuộc loss)
- nên mình có thể tối ưu 2 vòng, 1 vòng tạo mạch u, 1 vòng ráp mạch u vào

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import qtm.dag
import qiskit
%load_ext autoreload
%autoreload 2
torch.manual_seed(1000)
torch.cuda.manual_seed(1000)
np.random.seed(1000)
random.seed(1000)

qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.ry(np.pi/40, 1)
qc.rx(np.pi/40, 0)

qc1 = qiskit.QuantumCircuit(2)
qc.h(1)
qc.compose(qc1).draw()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


┌───┐     ┌──────────┐     
q_0: ┤ H ├──■──┤ Rx(π/40) ├─────
     └───┘┌─┴─┐├──────────┤┌───┐
q_1: ─────┤ X ├┤ Ry(π/40) ├┤ H ├
          └───┘└──────────┘└───┘

In [ ]:
import qtm.qcompilation
import numpy as np
import types
import qiskit
%load_ext autoreload
%autoreload 2

def reward_function(loss):
    return
def frozen_circuit(circuit):
    return

def unfrozen_circuit(frozen_circuit):
    return

def merge_circuit(qc1, qc2):
    return qc1.compose(qc2)

def Qdas(frozen_circuit, pool_theta, pool_operation):
    return
def qdart(initial_circuit, R, T, N):
    # neural_network 
    s = [None] * T
    a = [None] * T
    r = [None] * T
    pi = [None] * T
    policy_gradients = [None] * T
    s[0] = initial_circuit
    for i in range(0, N):
        for t in range(0, T):
            old_circuit = s[t]
            new_circuit = Qdas(frozen_circuit(old_circuit), pool_theta, pool_operation)
            a[t] = qtm.dag.circuit_to_scalar(s[t])
            if a[t] == 0:
                s[t+1] = unfrozen_circuit(old_circuit)
            else:
                s[t+1] = merge_circuit(unfrozen_circuit(old_circuit), new_circuit)
            r[t] = reward_function(loss)
            pi[t] = [s[t], a[t], r[t], s[t+1]]
        # Compute policy gradients
        # policy_gradients[i] = calculate_policy_gradient(pi[t])
    

In [5]:
import qtm.constant
gate_pool = qtm.constant.create_gate_pool(3)
print(gate_pool)

[('Identity', 0), ('Hadamard', 0), ('PauliX', 0), ('PauliY', 0), ('PauliZ', 0), ('RX', 0), ('RY', 0), ('RZ', 0), ('Identity', 1), ('Hadamard', 1), ('PauliX', 1), ('PauliY', 1), ('PauliZ', 1), ('RX', 1), ('RY', 1), ('RZ', 1), ('Identity', 2), ('Hadamard', 2), ('PauliX', 2), ('PauliY', 2), ('PauliZ', 2), ('RX', 2), ('RY', 2), ('RZ', 2), ('CNOT', 0, 1), ('CY', 0, 1), ('CZ', 0, 1), ('CRX', 0, 1), ('CRY', 0, 1), ('CRZ', 0, 1), ('CNOT', 0, 2), ('CY', 0, 2), ('CZ', 0, 2), ('CRX', 0, 2), ('CRY', 0, 2), ('CRZ', 0, 2), ('CNOT', 1, 0), ('CY', 1, 0), ('CZ', 1, 0), ('CRX', 1, 0), ('CRY', 1, 0), ('CRZ', 1, 0), ('CNOT', 1, 2), ('CY', 1, 2), ('CZ', 1, 2), ('CRX', 1, 2), ('CRY', 1, 2), ('CRZ', 1, 2), ('CNOT', 2, 0), ('CY', 2, 0), ('CZ', 2, 0), ('CRX', 2, 0), ('CRY', 2, 0), ('CRZ', 2, 0), ('CNOT', 2, 1), ('CY', 2, 1), ('CZ', 2, 1), ('CRX', 2, 1), ('CRY', 2, 1), ('CRZ', 2, 1)]
